### MLP Regressor on sampled Kilo-Sky data

In [1]:
import pandas as pd

In [2]:
import numpy as np

#### Environment

Set up environment and working directory

In [3]:
import os
workdir = os.environ['SCRATCH']
os.environ["BOSS_DATA_URL"] = 'http://dr12.sdss3.org'
os.environ["BOSS_LOCAL_ROOT"] = os.path.join(workdir,'sdss')
os.environ["BOSS_SAS_PATH"] = '/sas/dr12/boss'
os.environ["BOSS_REDUX_VERSION"]='v5_7_0'

In [4]:
os.chdir(workdir)

In [5]:
os.getcwd()

'/scratch/kunjias'

In [6]:
workdir

'/scratch/kunjias'

In [7]:
! ls

home			kilo_sky_r5_features.hdf5  sdss
kilo_sky_features.hdf5	qso.dat			   sdss.building


#### Data Import
Import data from HDF5 to pandas table.

In [8]:
kilo_sky_r5_table = pd.read_hdf('kilo_sky_r5_features.hdf5')

In [9]:
kilo_sky_r5_table.head()

,exposure_index,PLATE,MJD,XFOCAL,YFOCAL,FIBER,RA,DEC,OBJTYPE,AIRMASS_0,...,inverse_variance4119,inverse_variance4120,inverse_variance4121,inverse_variance4122,inverse_variance4123,inverse_variance4124,inverse_variance4125,inverse_variance4126,inverse_variance4127,inverse_variance4128
0,0,3844,55321,-93.327362,140.945984,566,180.40334,0.647407,SKY,0.0,...,4.611232e+06,3.338240e-33,0.0,0.000000e+00,0.000000,-2.216037e-35,3.064008e-06,0.0,0.000000e+00,-0.029867
1,1,3844,55321,-93.327362,140.945984,566,180.40334,0.647407,SKY,0.0,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000
2,2,3844,55321,-93.327362,140.945984,566,180.40334,0.647407,SKY,0.0,...,4.611232e+06,3.338240e-33,0.0,0.000000e+00,0.000000,-2.216037e-35,3.064008e-06,0.0,0.000000e+00,-0.029867
3,3,3844,55321,-93.327362,140.945984,566,180.40334,0.647407,SKY,0.0,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000
4,0,5399,55956,-119.386322,150.762390,640,185.17342,12.159917,SKY,0.0,...,7.174648e-43,-3.249857e+18,28537474.0,-1.079683e-15,12.765937,-1.816097e-37,9.403955e-38,0.0,1.356316e-19,0.000000


In [10]:
kilo_sky_r5_table.shape

(5593, 20654)

In [11]:
kilo_sky_r5_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5593 entries, 0 to 5592
Columns: 20654 entries, exposure_index to inverse_variance4128
dtypes: float32(20640), float64(9), int64(4), object(1)
memory usage: 441.0+ MB


In [12]:
list(kilo_sky_r5_table.columns.values)[8:13]

['OBJTYPE', 'AIRMASS_0', 'AIRMASS_1', 'AIRMASS_2', 'AIRMASS_3']

Drop the object data type column, and keep one airmass out of the four. `AIRMASS_4` is kept here.

In [13]:
kilo_sky_r5_table.drop(kilo_sky_r5_table.columns[np.arange(8,13)], axis=1,inplace=True)

In [14]:
kilo_sky_r5_table.shape

(5593, 20649)

The sky data table has 5 less columns now.

#### Target and Feature Data

In [15]:
len(list(kilo_sky_r5_table.filter(regex='total_flux')))

4128

In [16]:
kilo_sky_r5_table.filter(regex='total_flux').shape

(5593, 4128)

In [17]:
type(kilo_sky_r5_table.filter(regex='total_flux'))

pandas.core.frame.DataFrame

#### Data to prepare for training and testing set

In [18]:
X_r5 = kilo_sky_r5_table[kilo_sky_r5_table.columns.drop(list(kilo_sky_r5_table.filter(regex='total_flux')))]

In [19]:
X_r5.shape

(5593, 16521)

##### Target Data

In [20]:
y = kilo_sky_r5_table[list(kilo_sky_r5_table.filter(regex='total_flux'))]

In [21]:
y.shape

(5593, 4128)

#### Train Test Split

In [22]:
from sklearn.model_selection import train_test_split

20% are kept as testing set, the rest 80% are to be split for training and cross-validation.

Intermediate/ test split (Test Set)

In [23]:
X_intermediate, X_test, y_intermediate, y_test = train_test_split(X_r5,y,shuffle=True,test_size=0.2,random_state=5)

In [24]:
X_intermediate.shape

(4474, 16521)

In [25]:
X_test.shape

(1119, 16521)

Train / validation split (Train and Validation Set)

Training and validation data are split in a half-and-half ratio.

In [26]:
X_train, X_validation, y_train, y_validation = train_test_split(X_intermediate,y_intermediate,shuffle=False,test_size=0.5, random_state=5)

In [27]:
X_train.shape

(2237, 16521)

In [28]:
X_validation.shape

(2237, 16521)

Print proportions of datasets.

In [29]:
print('train: {}% | validation: {}% | test {}%'.format(round(y_train.shape[0]/y.shape[0],2),
                                                       round(y_validation.shape[0]/y.shape[0],2),
                                                       round(y_test.shape[0]/y.shape[0],2)))

train: 0.4% | validation: 0.4% | test 0.2%


There are some invalid values in the data sets. Fill them in by 0.

In [30]:
X_train_filled = X_train.fillna(value=0)

In [31]:
X_test_filled = X_test.fillna(value=0)

In [32]:
y_train_filled = y_train.fillna(value=0)

In [33]:
y_test_filled = y_test.fillna(value=0)

In [34]:
X_validation_filled= X_validation.fillna(value=0)

In [35]:
y_validation_filled = y_validation.fillna(value=0)

In [36]:
X_intermediate_filled = X_intermediate.fillna(value=0)

In [37]:
y_intermediate_filled = y_intermediate.fillna(value=0)

#### Data Preprocessing

Preprocess the data by z-score standardization

In [38]:
from sklearn.preprocessing import StandardScaler

In [39]:
scaler = StandardScaler()

In [40]:
scaler.fit(X_train_filled)

StandardScaler(copy=True, with_mean=True, with_std=True)

Apply same scaling to testing and validation data.

In [41]:
X_train_filled = scaler.transform(X_train_filled)

In [42]:
X_validation_filled = scaler.transform(X_validation_filled)

In [43]:
X_test_filled = scaler.transform(X_test_filled)

#### Traning the Model

In [44]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

We generate the neural network model with 4 layers and set the number of neurons to be 32, 64, 128, and 256. These neuron numbers are empirical choice from reconstructing spectrum for stellar parameters.

In [50]:
# Default 'relu' activation function
alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10]
print('-'*76)

for alpha in alphas:
    #fit the model
    mlp = MLPRegressor(hidden_layer_sizes=(32,64,128,256),alpha=alpha, max_iter=500,random_state=5)
    mlp.fit(X_train_filled, y_train_filled)
    
    #calculate errors
    new_train_error = mean_squared_error(y_train_filled, mlp.predict(X_train_filled))
    new_validation_error = mean_squared_error(y_validation_filled, mlp.predict(X_validation_filled))
    new_test_error = mean_squared_error(y_test_filled, mlp.predict(X_test_filled))
    
    #print error as report
    print('alpha: {:7} | train error: {:5} | val error: {:6} | test error: {}'.
          format(alpha,
                 round(new_train_error,3),
                 round(new_validation_error,3),
                 round(new_test_error,3)))

----------------------------------------------------------------------------
alpha:  0.0001 | train error: 1465.658 | val error: 2.4368096940856987e+61 | test error: 4.6614297444266455e+58
alpha:   0.001 | train error: 1908.115 | val error: 4.885584495986541e+61 | test error: 1.876512999927432e+58
alpha:    0.01 | train error: 1437.961 | val error: 2.9318359552636764e+61 | test error: 5.77559611026819e+58
alpha:     0.1 | train error: 1723.43 | val error: 8.874225370939916e+60 | test error: 2.2509093939530088e+58
alpha:       1 | train error: 1641.163 | val error: 2.3168180946171658e+61 | test error: 2.8273158933116783e+58
alpha:      10 | train error: 1833.456 | val error: 1.2982299268392093e+61 | test error: 4.1251906916446885e+58


In [47]:
# 'lbfgs' activation function
alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10]
print('-'*76)

for alpha in alphas:
    #fit the model
    mlp = MLPRegressor(hidden_layer_sizes=(32,64,128,256),solver='lbfgs', alpha=alpha, max_iter=500,random_state=5)
    mlp.fit(X_train_filled, y_train_filled)
    
    #calculate errors
    new_train_error = mean_squared_error(y_train_filled, mlp.predict(X_train_filled))
    new_validation_error = mean_squared_error(y_validation_filled, mlp.predict(X_validation_filled))
    new_test_error = mean_squared_error(y_test_filled, mlp.predict(X_test_filled))
    
    #print error as report
    print('alpha: {:7} | train error: {:5} | val error: {:6} | test error: {}'.
          format(alpha,
                 round(new_train_error,3),
                 round(new_validation_error,3),
                 round(new_test_error,3)))

----------------------------------------------------------------------------
alpha:  0.0001 | train error: 3884.992 | val error: 1.9732537381890759e+58 | test error: 1.6148733081323083e+54
alpha:   0.001 | train error: 3884.992 | val error: 1.9732537381890759e+58 | test error: 1.6148733081323083e+54
alpha:    0.01 | train error: 3884.992 | val error: 1.9732537381890759e+58 | test error: 1.6148733081323083e+54
alpha:     0.1 | train error: 3884.992 | val error: 1.9732537381890759e+58 | test error: 1.6148733081323083e+54
alpha:       1 | train error: 3884.992 | val error: 1.9732537381890759e+58 | test error: 1.6148733081323083e+54
alpha:      10 | train error: 3884.992 | val error: 1.9732537381890759e+58 | test error: 1.6148733081323083e+54


In [48]:
alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10]
print('-'*76)

for alpha in alphas:
    #fit the model
    mlp = MLPRegressor(hidden_layer_sizes=(32,64,128,256),solver='adam', alpha=alpha, max_iter=500,random_state=5)
    mlp.fit(X_train_filled, y_train_filled)
    
    #calculate errors
    new_train_error = mean_squared_error(y_train_filled, mlp.predict(X_train_filled))
    new_validation_error = mean_squared_error(y_validation_filled, mlp.predict(X_validation_filled))
    new_test_error = mean_squared_error(y_test_filled, mlp.predict(X_test_filled))
    
    #print error as report
    print('alpha: {:7} | train error: {:5} | val error: {:6} | test error: {}'.
          format(alpha,
                 round(new_train_error,3),
                 round(new_validation_error,3),
                 round(new_test_error,3)))

----------------------------------------------------------------------------
alpha:  0.0001 | train error: 1465.658 | val error: 2.4368096940856987e+61 | test error: 4.6614297444266455e+58
alpha:   0.001 | train error: 1908.115 | val error: 4.885584495986541e+61 | test error: 1.876512999927432e+58
alpha:    0.01 | train error: 1437.961 | val error: 2.9318359552636764e+61 | test error: 5.77559611026819e+58
alpha:     0.1 | train error: 1723.43 | val error: 8.874225370939916e+60 | test error: 2.2509093939530088e+58
alpha:       1 | train error: 1641.163 | val error: 2.3168180946171658e+61 | test error: 2.8273158933116783e+58
alpha:      10 | train error: 1833.456 | val error: 1.2982299268392093e+61 | test error: 4.1251906916446885e+58


In [46]:
alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10]
print('-'*76)

for alpha in alphas:
    #fit the model
    mlp = MLPRegressor(hidden_layer_sizes=(100,100,100,100),alpha=alpha, max_iter=500,random_state=5)
    mlp.fit(X_train_filled, y_train_filled)
    
    #calculate errors
    new_train_error = mean_squared_error(y_train_filled, mlp.predict(X_train_filled))
    new_validation_error = mean_squared_error(y_validation_filled, mlp.predict(X_validation_filled))
    new_test_error = mean_squared_error(y_test_filled, mlp.predict(X_test_filled))
    
    #print error as report
    print('alpha: {:7} | train error: {:5} | val error: {:6} | test error: {}'.
          format(alpha,
                 round(new_train_error,3),
                 round(new_validation_error,3),
                 round(new_test_error,3)))

----------------------------------------------------------------------------
alpha:  0.0001 | train error: 1046.183 | val error: 4.910851715397637e+61 | test error: 1.1306690817331811e+59
alpha:   0.001 | train error: 1582.679 | val error: 3.9365363524195863e+61 | test error: 1.1290542798949676e+59
alpha:    0.01 | train error: 1215.134 | val error: 5.39001782006778e+61 | test error: 1.3749317483976003e+59
alpha:     0.1 | train error: 1289.846 | val error: 3.46498809636416e+61 | test error: 1.8400203187815024e+59
alpha:       1 | train error: 1180.225 | val error: 2.923149860414411e+61 | test error: 2.016448721967014e+59
alpha:      10 | train error: 1184.365 | val error: 4.162896674919318e+61 | test error: 1.8047803112106128e+59


In [62]:
alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10]
print('-'*76)

for alpha in alphas:
    #fit the model
    mlp = MLPRegressor(hidden_layer_sizes=(100,100,100),activation = 'identity',alpha=alpha, max_iter=500,random_state=5)
    mlp.fit(X_train_filled, y_train_filled)
    
    #calculate errors
    new_train_error = mean_squared_error(y_train_filled, mlp.predict(X_train_filled))
    new_validation_error = mean_squared_error(y_validation_filled, mlp.predict(X_validation_filled))
    new_test_error = mean_squared_error(y_test_filled, mlp.predict(X_test_filled))
    
    #print error as report
    print('alpha: {:7} | train error: {:5} | val error: {:6} | test error: {}'.
          format(alpha,
                 round(new_train_error,3),
                 round(new_validation_error,3),
                 round(new_test_error,3)))

----------------------------------------------------------------------------
alpha:  0.0001 | train error: 2373.914 | val error: 5.709254436786455e+61 | test error: 7.085585588498432e+47
alpha:   0.001 | train error: 2187.446 | val error: 2.7165360979457124e+61 | test error: 2.2152728630486337e+47
alpha:    0.01 | train error: 2186.542 | val error: 2.71555853213082e+61 | test error: 2.214996462276027e+47
alpha:     0.1 | train error: 2187.617 | val error: 2.7188017771218483e+61 | test error: 2.217232340725158e+47
alpha:       1 | train error: 2328.101 | val error: 5.206340408406818e+61 | test error: 6.738853675607587e+47
alpha:      10 | train error: 2188.39 | val error: 2.7538136043467724e+61 | test error: 2.2328669174309954e+47


In [49]:
alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10]
print('-'*76)

for alpha in alphas:
    #fit the model
    mlp = MLPRegressor(hidden_layer_sizes=(100,100,100),activation = 'logistic',alpha=alpha, max_iter=500,random_state=5)
    mlp.fit(X_train_filled, y_train_filled)
    
    #calculate errors
    new_train_error = mean_squared_error(y_train_filled, mlp.predict(X_train_filled))
    new_validation_error = mean_squared_error(y_validation_filled, mlp.predict(X_validation_filled))
    new_test_error = mean_squared_error(y_test_filled, mlp.predict(X_test_filled))
    
    #print error as report
    print('alpha: {:7} | train error: {:5} | val error: {:6} | test error: {}'.
          format(alpha,
                 round(new_train_error,3),
                 round(new_validation_error,3),
                 round(new_test_error,3)))

----------------------------------------------------------------------------


/project/6016730/mlhaa/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


alpha:  0.0001 | train error: 3303.899 | val error: 9191.024 | test error: 3779.058
alpha:   0.001 | train error: 3301.081 | val error: 9177.256 | test error: 3796.991


/project/6016730/mlhaa/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


alpha:    0.01 | train error: 3333.287 | val error: 9218.692 | test error: 3806.141
alpha:     0.1 | train error: 3312.655 | val error: 9191.421 | test error: 3912.313
alpha:       1 | train error: 3547.31 | val error: 9390.383 | test error: 3728.425
alpha:      10 | train error: 3722.895 | val error: 9564.395 | test error: 3820.018


In [57]:
alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10]
print('-'*76)

for alpha in alphas:
    #fit the model
    mlp = MLPRegressor(hidden_layer_sizes=(100,100,100),activation = 'tanh',alpha=alpha, max_iter=500,random_state=5)
    mlp.fit(X_train_filled, y_train_filled)
    
    #calculate errors
    new_train_error = mean_squared_error(y_train_filled, mlp.predict(X_train_filled))
    new_validation_error = mean_squared_error(y_validation_filled, mlp.predict(X_validation_filled))
    new_test_error = mean_squared_error(y_test_filled, mlp.predict(X_test_filled))
    
    #print error as report
    print('alpha: {:7} | train error: {:5} | val error: {:6} | test error: {}'.
          format(alpha,
                 round(new_train_error,3),
                 round(new_validation_error,3),
                 round(new_test_error,3)))

----------------------------------------------------------------------------


/project/6016730/mlhaa/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


alpha:  0.0001 | train error: 3352.883 | val error: 9218.59 | test error: 4066.224


/project/6016730/mlhaa/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


alpha:   0.001 | train error: 3114.128 | val error: 9028.718 | test error: 3923.971
alpha:    0.01 | train error: 3144.184 | val error: 9065.596 | test error: 4078.906
alpha:     0.1 | train error: 3261.281 | val error: 9153.053 | test error: 4333.866
alpha:       1 | train error: 3544.282 | val error: 9400.227 | test error: 3885.013
alpha:      10 | train error: 3722.278 | val error: 9586.258 | test error: 3952.441


Auto-sklearn

In [66]:
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics